## Web extraction from info.dengue data

> autor: BragatteMAS

> updated: 20231211 

#### Refs

https://info.dengue.mat.br/

https://info.dengue.mat.br/services/api

In [37]:
import pandas as pd ## for dataframes
from datetime import datetime, timedelta ## for dealing with time

In [38]:
## List of geocodes for Brazilian states
geocodes = [
    1200401, # Acre
    2704302, # Alagoas
    1302603, # Amazonas
    1600303, # Amapá
    2927408, # Bahia
    2304400, # Ceará
    5300108, # Distrito Federal
    3205309, # Espírito Santo
    5208707, # Goiás
    2111300, # Maranhão
    5103403, # Mato Grosso
    5002704, # Mato Grosso do Sul
    3106200, # Minas Gerais
    1501402, # Pará
    2507507, # Paraíba
    4115200, # Paraná
    2611606, # Pernambuco
    2207702, # Piauí
    3304557, # Rio de Janeiro
    2408102, # Rio Grande do Norte
    4314902, # Rio Grande do Sul
    1100023, # Rondônia
    1400100, # Roraima
    4205407, # Santa Catarina
    3550308, # São Paulo
    2800308, # Sergipe
    1721000  # Tocantins
]

## Mapping of geocodes to state names
geocode_to_state = {
    1200401: "Acre",
    2704302: "Alagoas",
    1302603: "Amazonas",
    1600303: "Amapá",
    2927408: "Bahia",
    2304400: "Ceará",
    5300108: "Distrito Federal",
    3205309: "Espírito Santo",
    5208707: "Goiás",
    2111300: "Maranhão",
    5103403: "Mato Grosso",
    5002704: "Mato Grosso do Sul",
    3106200: "Minas Gerais",
    1501402: "Pará",
    2507507: "Paraíba",
    4115200: "Paraná",
    2611606: "Pernambuco",
    2207702: "Piauí",
    3304557: "Rio de Janeiro",
    2408102: "Rio Grande do Norte",
    4314902: "Rio Grande do Sul",
    1100023: "Rondônia",
    1400100: "Roraima",
    4205407: "Santa Catarina",
    3550308: "São Paulo",
    2800308: "Sergipe",
    1721000: "Tocantins"
}

## Mapping of geocodes to state codes names
geocode_to_uf = {
    1200401: "AC",
    2704302: "AL",
    1302603: "AM",
    1600303: "AP",
    2927408: "BA",
    2304400: "CE",
    5300108: "DF",
    3205309: "ES",
    5208707: "GO",
    2111300: "MA",
    5103403: "MT",
    5002704: "MS",
    3106200: "MG",
    1501402: "PA",
    2507507: "PB",
    4115200: "PR",
    2611606: "PE",
    2207702: "PI",
    3304557: "RJ",
    2408102: "RN",
    4314902: "RS",
    1100023: "RO",
    1400100: "RR",
    4205407: "SC",
    3550308: "SP",
    2800308: "SE",
    1721000: "TO"
}

## Mapping of state codes to geocodes
uf_to_region = {
    "AC": "Norte", "AP": "Norte", "AM": "Norte", "PA": "Norte", "RO": "Norte", "RR": "Norte", "TO": "Norte",
    "AL": "Nordeste", "BA": "Nordeste", "CE": "Nordeste", "MA": "Nordeste", "PB": "Nordeste",
    "PE": "Nordeste", "PI": "Nordeste", "RN": "Nordeste", "SE": "Nordeste",
    "DF": "Centro-Oeste", "GO": "Centro-Oeste", "MT": "Centro-Oeste", "MS": "Centro-Oeste",
    "ES": "Sudeste", "MG": "Sudeste", "RJ": "Sudeste", "SP": "Sudeste",
    "PR": "Sul", "RS": "Sul", "SC": "Sul"
}

In [39]:
## Columns to be selected from infodengue to compose infodenv_cases (ITpS filter)
new_columns = ['data_iniSE', 'SE', 'casos_est', 'casos_est_min', 'casos_est_max', 'casos']

## Initialize an empty DataFrame to store the selected data
infodenv_cases = pd.DataFrame()

## Loop over each geocode, fetch data, select columns, add state column and concatenate
for geocode in geocodes:
    url = f'https://info.dengue.mat.br/api/alertcity?geocode={geocode}&disease=dengue&format=csv&ew_start=1&ew_end=50&ey_start=2022&ey_end=2023'
    infodengue = pd.read_csv(url)
    selected_data = infodengue[new_columns]
    selected_data['state_code'] = geocode_to_uf[geocode]  # Add a column for the state_code
    selected_data['state'] = geocode_to_state[geocode]  # Add a column for the state name
    selected_data['region'] = uf_to_region[geocode_to_uf[geocode]]  # Add a column for the region
    infodenv_cases = pd.concat([infodengue, selected_data])

## Reset index after concatenating
infodenv_cases.reset_index(drop=True, inplace=True)

## Display the first few rows of the new DataFrame
infodenv_cases.head()

/var/folders/gt/c4rqf02d5m731j41w9p5mt2h0000gp/T/ipykernel_33206/1442705092.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['state_code'] = geocode_to_uf[geocode]  # Add a column for the state_code
/var/folders/gt/c4rqf02d5m731j41w9p5mt2h0000gp/T/ipykernel_33206/1442705092.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['state'] = geocode_to_state[geocode]  # Add a column for the state name
/var/folders/gt/c4rqf02d5m731j41w9p5mt2h0000gp/T/ipykernel_33206/1442705092.py:1

,data_iniSE,SE,casos_est,casos_est_min,casos_est_max,casos,state_code,state,region,data_fimSE
0,2023-11-05,202345,359.0,99,1291.0,5,AC,Acre,Norte,2023-11-11
1,2023-10-29,202344,325.0,202,753.0,169,AC,Acre,Norte,2023-11-04
2,2023-10-22,202343,249.0,219,340.0,210,AC,Acre,Norte,2023-10-28
3,2023-10-15,202342,174.0,166,197.0,167,AC,Acre,Norte,2023-10-21
4,2023-10-08,202341,108.0,105,115.0,108,AC,Acre,Norte,2023-10-14


In [40]:
## Display the columns of the original DataFrame
print(infodengue.columns)

Index(['data_iniSE', 'SE', 'casos_est', 'casos_est_min', 'casos_est_max',
       'casos', 'p_rt1', 'p_inc100k', 'Localidade_id', 'nivel', 'id',
       'versao_modelo', 'tweet', 'Rt', 'pop', 'tempmin', 'umidmax',
       'receptivo', 'transmissao', 'nivel_inc', 'umidmed', 'umidmin',
       'tempmed', 'tempmax', 'casprov', 'casprov_est', 'casprov_est_min',
       'casprov_est_max', 'casconf', 'notif_accum_year'],
      dtype='object')


Corrigindo data da semana

In [28]:
## Function to add 6 days to a date string
def add_days_to_date(date_str):
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    end_date_obj = date_obj + timedelta(days=6)
    return end_date_obj.strftime('%Y-%m-%d')

## Apply the function to the column 'data_iniSE' and store the result in a new column 'data_fimSE'
infodenv_cases['data_fimSE'] = infodenv_cases['data_iniSE'].apply(add_days_to_date)

## Display the first few rows of the new DataFrame
infodenv_cases.head()

,data_iniSE,SE,casos_est,casos_est_min,casos_est_max,casos,state_code,state,region,data_fimSE
0,2023-11-05,202345,359.0,99,1291.0,5,AC,Acre,Norte,2023-11-11
1,2023-10-29,202344,325.0,202,753.0,169,AC,Acre,Norte,2023-11-04
2,2023-10-22,202343,249.0,219,340.0,210,AC,Acre,Norte,2023-10-28
3,2023-10-15,202342,174.0,166,197.0,167,AC,Acre,Norte,2023-10-21
4,2023-10-08,202341,108.0,105,115.0,108,AC,Acre,Norte,2023-10-14


In [29]:
## Pivot the DataFrame cases
pivoted_df_cases = infodenv_cases.pivot_table(
    index='state_code',             # Set 'Estado' as the index
    columns='data_fimSE',       # Set 'data_iniSE' as the columns
    values='casos',         # Choose the values to display, e.g., 'casos'
    aggfunc='sum'               # Aggregate function if there are multiple entries for the same index/column pair
)

## Pivot the DataFrame cases_est
pivoted_df_cases_est = infodenv_cases.pivot_table(
    index='state_code',             # Set 'Estado' as the index
    columns='data_fimSE',       # Set 'data_iniSE' as the columns
    values='casos_est',         # Choose the values to display, e.g., 'casos_est'
    aggfunc='sum'               # Aggregate function if there are multiple entries for the same index/column pair
)

In [31]:
## Display the pivoted DataFrame from cases
# pivoted_df_casos.tail()

In [32]:
## Display the pivoted DataFrame from cases estimated
# pivoted_df_casos_est.tail()

data_fimSE,2022-01-08,2022-01-15,2022-01-22,2022-01-29,2022-02-05,2022-02-12,2022-02-19,2022-02-26,2022-03-05,2022-03-12,...,2023-09-09,2023-09-16,2023-09-23,2023-09-30,2023-10-07,2023-10-14,2023-10-21,2023-10-28,2023-11-04,2023-11-11
state_code,,,,,,,,,,,,,,,,,,,,,
RS,5.0,3.0,6.0,2.0,3.0,3.0,14.0,26.0,61.0,227.0,...,26.0,35.0,30.0,20.0,22.0,15.0,17.0,16.0,15.0,14.0
SC,44.0,32.0,22.0,22.0,16.0,32.0,21.0,50.0,50.0,112.0,...,154.0,175.0,178.0,133.0,162.0,139.0,124.0,148.0,150.0,144.0
SE,2.0,12.0,7.0,2.0,9.0,7.0,19.0,7.0,18.0,10.0,...,68.0,74.0,67.0,70.0,57.0,72.0,74.0,61.0,61.0,56.0
SP,327.0,253.0,366.0,325.0,289.0,310.0,322.0,342.0,504.0,627.0,...,570.0,646.0,781.0,703.0,832.0,668.0,681.0,704.0,639.0,736.0
TO,1248.0,1030.0,946.0,901.0,892.0,798.0,695.0,542.0,545.0,592.0,...,70.0,64.0,47.0,30.0,37.0,79.0,73.0,47.0,63.0,54.0


In [30]:
# Save the pivoted DataFrame from cases to a TSV file
pivoted_df_cases.to_csv('casos_denv.tsv', sep='\t')

# Save the pivoted DataFrame from cases estimated to a TSV file
pivoted_df_cases_est.to_csv('casosest_denv.tsv', sep='\t')

## Save the infodenv_cases fiile to a CSV file
infodenv_cases.to_csv('infodengue_casos_filtro.csv', index=False)

## Site e.g.

https://info.dengue.mat.br/services/api/doc

In [6]:
# 3304557 = Rio de Janeiro city geocode
url = 'https://info.dengue.mat.br/api/alertcity?geocode=3304557&disease=dengue&format=csv&ew_start=1&ew_end=50&ey_start=2022&ey_end=2023'
search_filter = (
    'geocode=3304557&disease=dengue&format=csv&' +
    'ew_start=1&ew_end=50&ey_start=2022&ey_end=2023'
)
df = pd.read_csv('%s?%s' % (url, search_filter))
print(url)
df.head()

https://info.dengue.mat.br/api/alertcity?geocode=3304557&disease=dengue&format=csv&ew_start=1&ew_end=50&ey_start=2022&ey_end=2023


,data_iniSE,SE,casos_est,casos_est_min,casos_est_max,casos,p_rt1,p_inc100k,Localidade_id,nivel,...,umidmed,umidmin,tempmed,tempmax,casprov,casprov_est,casprov_est_min,casprov_est_max,casconf,notif_accum_year
0,2023-11-05,202345,796.0,481,1516,248,1.000000,11.796411,0,2,...,70.412289,67.700949,23.256250,23.900000,NaN,NaN,NaN,NaN,NaN,45783
1,2023-10-29,202344,480.0,421,590,349,0.880848,7.113414,0,2,...,79.559109,73.389470,25.595238,27.333333,NaN,NaN,NaN,NaN,NaN,45783
2,2023-10-22,202343,459.0,435,495,407,0.832322,6.802202,0,2,...,80.318641,77.000410,23.244048,23.809524,NaN,NaN,NaN,NaN,NaN,45783
3,2023-10-15,202342,470.0,456,490,443,0.822390,6.965218,0,2,...,81.240760,78.645457,22.581250,23.000000,NaN,NaN,NaN,NaN,NaN,45783
4,2023-10-08,202341,382.0,373,396,366,0.001660,5.661092,0,2,...,85.822647,83.576571,23.726191,24.000000,NaN,NaN,NaN,NaN,NaN,45783


In [22]:
## extract data from RJ
new_columns = ['data_iniSE', 'SE', 'casos_est', 'casos_est_min', 'casos_est_max', 'casos']
new_df = infodengue[new_columns]
new_df.head()


,data_iniSE,SE,casos_est,casos_est_min,casos_est_max,casos
0,2023-11-05,202345,54.0,35,218,33
1,2023-10-29,202344,63.0,59,90,59
2,2023-10-22,202343,47.0,46,54,46
3,2023-10-15,202342,73.0,73,76,73
4,2023-10-08,202341,79.0,79,81,79
